In [1]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load the data

filepath_finance_data = '../Data/Datasets/updated_finance_data.csv'
finance_data = pd.read_csv(filepath_finance_data)


In [ ]:
# Checking the data
# ordering all columns in alphabetical order

finance_data = finance_data.reindex(sorted(finance_data.columns), axis=1)

# for easier viewing, I will make sure all float values are rounded to 2 decimal places and have commas

pd.options.display.float_format = '{:,.2f}'.format

# sorting the finance data by symbol (ascending) and date_x (ascending)

finance_data = finance_data.sort_values(by=['symbol', 'date_x'], ascending=[True, True])

# removing countries that create noise in the data

country2remove = ['IN', 'JP', 'CN', 'BR', 'TW', 'DK'] # removed as it looks like some of the data was not accurate for these countries

finance_data = finance_data[~finance_data['country'].isin(country2remove)]

# viewing the first 10 rows of the data

finance_data.head(10)


In [ ]:
# creating a smaller dataset for the machine learning model. First by defining the columns to keep and second by making a copy of the data and filtering for the columns to keep.

kpi2keep = ['companyName', 'date_x', 'fiscalDateEnding', 'sector', 'symbol',
            'revenuePerShare', 'capexPerShare', 'operatingCashFlowPerShare_x', 'epsEstimated',
            'grossProfitMargin', 'returnOnEquity', 'freeCashFlowPerShare_x', 'stockPrice', 'peRatio',
            'bookValuePerShare', 'cashPerShare_x',]